<a href="https://colab.research.google.com/github/lzvtaa/odr-test234/blob/main/_downloads/af0caf6d7af0dda755f4c9d7af9ccc2c/quickstart_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elizaveta Petrukhina

petrukhe@oregonstate.edu

AI 530

11/7/2025

HW 6

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
torch.manual_seed(0)

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 13.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 208kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.90MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 19.4MB/s]


In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
import torch
from torch.utils.data import Subset

torch.manual_seed(0)
idx = torch.randperm(len(test_data))[:100]
test_loader_100 = DataLoader(Subset(test_data, idx), batch_size=100, shuffle=False)

------------------------------------------------------------------------


Creating Models
===============

To define a neural network in PyTorch, we create a class that inherits
from
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
We define the layers of the network in the `__init__` function and
specify how data will pass through the network in the `forward`
function. To accelerate operations in the neural network, we move it to
the
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [6]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

import torch.nn as nn

class Arch1(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 512), nn.ReLU(),
            nn.Linear(512, 512),   nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

class Arch2(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 512), nn.ReLU(),
            nn.Linear(512, 512),   nn.ReLU(),
            nn.Linear(512, 512),   nn.ReLU(),  # extra hidden layer
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

class Arch3(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 512), nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

class Arch4(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 512), nn.ReLU(),
            nn.Linear(512, 1024),  nn.ReLU(),
            nn.Linear(1024, 512),  nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        return self.layers(x)

def get_model(name: str):
    cls = {"Arch1": Arch1, "Arch2": Arch2, "Arch3": Arch3, "Arch4": Arch4}[name]
    return cls().to(device)

Using cpu device


------------------------------------------------------------------------


Optimizing the Model Parameters
===============================

To train a model, we need a [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
import time

def train_one_model(model, epochs=5, lr=1e-3):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    if device == "cuda":
        torch.cuda.synchronize()
    start_time = time.time()

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)

    if device == "cuda":
        torch.cuda.synchronize()
    end_time = time.time()
    runtime = end_time - start_time

    print(f"Total training time: {runtime:.2f} seconds")
    return runtime


@torch.no_grad()
def mcr_on_100(model):
    model.eval()
    total, wrong = 0, 0
    for X, y in test_loader_100:
        X, y = X.to(device), y.to(device)
        preds = model(X).argmax(1)
        total += y.size(0)
        wrong += (preds != y).sum().item()
    mcr = wrong / total
    print(f"MCR = {mcr:.3f} ({wrong}/{total} wrong)")
    return mcr

In [11]:
ARCHS = {
    "Arch1": Arch1,
    "Arch2": Arch2,
    "Arch3": Arch3,
    "Arch4": Arch4,
}

results = []
EPOCHS = 15

for name, ModelClass in ARCHS.items():
    print(f"\n=== Running {name} ===")
    model = ModelClass().to(device)
    runtime = train_one_model(model, epochs=EPOCHS, lr=1e-3)
    mcr = mcr_on_100(model)
    results.append((name, EPOCHS, mcr, runtime))

print("\n=== Summary ===")
print(f"{'Model':<6} {'Epochs':<6} {'MCR':<6} {'Time(s)':<8}")
for name, ep, mcr, time_s in results:
    print(f"{name:<6} {ep:<6} {mcr:<6.3f} {time_s:<8.1f}")


=== Running Arch1 ===
Epoch 1
-------------------------------
loss: 2.296649  [   64/60000]
loss: 2.284453  [ 6464/60000]
loss: 2.275748  [12864/60000]
loss: 2.266878  [19264/60000]
loss: 2.235930  [25664/60000]
loss: 2.237536  [32064/60000]
loss: 2.220459  [38464/60000]
loss: 2.200824  [44864/60000]
loss: 2.202283  [51264/60000]
loss: 2.199637  [57664/60000]
Test Error: 
 Accuracy: 49.0%, Avg loss: 2.151510 

Epoch 2
-------------------------------
loss: 2.129921  [   64/60000]
loss: 2.123567  [ 6464/60000]
loss: 2.114925  [12864/60000]
loss: 2.080405  [19264/60000]
loss: 2.050580  [25664/60000]
loss: 2.031380  [32064/60000]
loss: 1.982722  [38464/60000]
loss: 1.930877  [44864/60000]
loss: 1.944222  [51264/60000]
loss: 1.905746  [57664/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 1.874461 

Epoch 3
-------------------------------
loss: 1.896966  [   64/60000]
loss: 1.744320  [ 6464/60000]
loss: 1.754887  [12864/60000]
loss: 1.762450  [19264/60000]
loss: 1.692015  [25664/60000]
los

------------------------------------------------------------------------


Saving Models
=============

A common way to save a model is to serialize the internal state
dictionary (containing the model parameters).


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
==============

The process for loading a model includes re-creating the model structure
and loading the state dictionary into it.


In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
